# Hands-On Chatbot OWB 06-2019

## Useful shortcuts:
### shift + enter: execute cell
### esc + m: enter markdown mode
### enter: edit mode
### "#,##,### Headline" headlines text

## Have fun, and don't hesistate to ask questions!

In [ ]:
import matplotlib
%matplotlib inline

# Download and link language model

In [ ]:
import sys
python = sys.executable

In [ ]:
!{python} -m spacy download --user en_core_web_md;
!{python} -m spacy link en_core_web_md en --force;

# Import required components
- Natural Language Understanding
- Dialog Management (Rasa Core)
- SpaCy Language Model

In [ ]:
import rasa_nlu

In [ ]:
import rasa_core

In [ ]:
import spacy

# Train the NLU component

### NLU training data

In [ ]:
nlu_md = """
## intent:greet
- hey
- hello there

## intent:goodbye
- good bye

## intent:mood_affirm
- yes

## intent:mood_deny
- no

## intent:mood_great
- great

## intent:mood_unhappy
- I am sad
- I am very sad. I need a [cat](group:cats) picture.
- Extremely sad. Only the cute [doggo](group:shibes) pics can make me feel better.

## intent: inform
- A [dog](group:shibes)
- [dog](group:shibes)
- of a [dog](group:shibes)
"""

%store nlu_md > nlu.md

## Rasa_NLU configuration

In [ ]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms
""" 

%store config > config.yml

## Train the NLU model

In [ ]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

## Evaluate the model (exemplary)

helper function pretty print "pprint"

In [ ]:
import json
def pprint(o):
    print(json.dumps(o, indent=2))

### Random tests

In [ ]:
pprint(interpreter.parse("something you want to test"))

In [ ]:
pprint(interpreter.parse("I want pictures of cats"))

## Evaluate the NLU with test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [ ]:
from rasa_nlu.evaluate import run_evaluation

run_evaluation("nlu.md", model_directory)

# Teach the chatbot how to respond

## Write Stories

In [ ]:
stories_md = """
## happy path               <!-- name of the story - just for debugging -->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_goodbye
  
## sad path 1               <!-- this is already the start of the next story -->
* greet
  - utter_greet             <!-- action the bot should execute -->
* mood_unhappy
  - utter_ask_picture
* inform{"animal":"dog"}  
  - action_retrieve_image
  - utter_did_that_help
* mood_affirm
  - utter_happy

##
## Task: Write some stories!
##

## say goodbye
* goodbye
  - utter_goodbye

## fallback
- utter_unclear

"""

%store stories_md > stories.md

## Define the domain

In [ ]:
domain_yml = """
intents:
- greet
- goodbye
- mood_affirm
- mood_deny
- mood_great
- mood_unhappy
- inform

slots:
  group:
    type: text
    
entities:
- group

actions:
- utter_greet
- utter_did_that_help
- utter_happy
- utter_goodbye
- utter_unclear
- utter_ask_picture
- __main__.ApiAction

templates:
  utter_greet:
  - text: "Hey! How are you?"
  ##add more greetings!

  utter_did_that_help:
  - text: "Did that help you?"
  ##add more variations!

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  ##add more
  
  utter_happy:
  - text: "Great carry on!"
  ##add more

  utter_goodbye:
  - text: "Bye"
  ##add more
  
  utter_ask_picture:
  - text: "To cheer you up, I can show you a cute picture of a dog, cat or a bird. Which one do you choose?"
  ##add more
"""

%store domain_yml > domain.yml

## Custom Action
### Helper Function to get pictures of animals

This is a custom action that requests pictures of dogs, cats or birds

In [ ]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet
from IPython.core.display import Image, display

import requests

class ApiAction(Action):
    def name(self):
        return "action_retrieve_image"

    def run(self, dispatcher, tracker, domain):
        
        group = tracker.get_slot('group')
        
        r = requests.get('http://shibe.online/api/{}?count=1&urls=true&httpsUrls=true'.format(group))
        response = r.content.decode()
        response = response.replace('["',"")
        response = response.replace('"]',"")
   
        
        #display(Image(response[0], height=550, width=520))
        dispatcher.utter_message("Here is something to cheer you up: {}".format(response))

## How do the training data dialogues look like?

i.e. our stories

In [ ]:
import pygraphviz

In [ ]:
from IPython.display import Image
from rasa_core.agent import Agent

agent = Agent('domain.yml')
agent.visualize("stories.md", "story_graph.png", max_history=2)
Image(filename="story_graph.png")

## Train the dialogue model

MemoizationPolicy: just memorizes the conversations in your training data. (It predicts the next action with confidence 1.0 if this exact conversation exists in the training data, otherwise it predicts None with confidence 0.0.)

KerasPolicy: uses a neural network implemented in Keras to select the next action. The default architecture is based on an LSTM (a recurrent neural network that has some sort of memory ("long short term memory") and are therefore good for time series data)

In [ ]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(
    training_data,
    validation_split=0.0,
    epochs=200
)

agent.persist('models/dialogue')

# Talk to the chatbot

In [ ]:
#Starting the Bot

from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

In [ ]:
Hey